In [2]:
import sys
sys.path.append("./work/streamify")

from operators.streaming import SparkStreaming
import pyspark.sql.functions as F

In [3]:
spark = SparkStreaming.get_instance(app_name="Streamify Spark Streaming")

In [21]:
spark

In [6]:
delta_df = spark.read.format("delta").load("s3a://lakehouse/streaming/streamify/rental")
print(delta_df.count())
delta_df.printSchema()

28
root
 |-- rental_id: integer (nullable = true)
 |-- rental_date: string (nullable = true)
 |-- inventory_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- return_date: string (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- last_update: string (nullable = true)



In [5]:
delta_df.show(5)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|   281488|2010-01-02 13:17:17|        1882|        114|2010-01-04 12:11:04|     124|2010-01-01 19:22:41|
|   281493|2010-01-03 00:13:16|        2080|        529|2010-01-05 21:48:32|      76|2010-01-02 01:30:56|
|   281502|2010-01-03 10:45:13|        2517|        599|2010-01-05 10:08:02|       3|2010-01-02 10:00:52|
|   281476|2010-01-01 07:47:23|        3430|        167|2010-01-03 04:21:21|     122|2010-01-01 04:05:03|
|   281477|2010-01-01 09:15:59|        1161|        446|2010-01-05 07:47:23|       1|2010-01-01 05:08:26|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 5 rows



In [6]:
customer_df = spark.read.csv("s3a://lakehouse/csv/customer.csv", header=True, inferSchema=True, samplingRatio=0.1)
address_df = spark.read.csv("s3a://lakehouse/csv/address.csv", header=True, inferSchema=True, samplingRatio=0.1)
city_df = spark.read.csv("s3a://lakehouse/csv/city.csv", header=True, inferSchema=True, samplingRatio=0.1)
country_df = spark.read.csv("s3a://lakehouse/csv/country.csv", header=True, inferSchema=True, samplingRatio=0.1)

In [10]:
spark.sql("CREATE DATABASE streamify").show()

++
||
++
++



In [11]:
customer_df.write.format("delta").mode("overwrite").saveAsTable("streamify.customer")
address_df.write.format("delta").mode("overwrite").saveAsTable("streamify.address")
city_df.write.format("delta").mode("overwrite").saveAsTable("streamify.city")
country_df.write.format("delta").mode("overwrite").saveAsTable("streamify.country")

In [14]:
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|streamify|  address|      false|
|streamify|     city|      false|
|streamify|  country|      false|
|streamify| customer|      false|
+---------+---------+-----------+



In [17]:
spark.stop()